In [ ]:
%matplotlib inline
import pandas as pd
import numpy as np
import xgboost as xgb


In [3]:
members = pd.read_csv("/mnt/volume-sgp1-02/members.csv",parse_dates=['registration_init_time','expiration_date'], dtype={'city': np.int8, 'bd': np.int16, 'registered_via': np.int8})
train = pd.read_csv('/mnt/volume-sgp1-02/train.csv', dtype={'is_churn' : np.int8})
transactions = pd.read_csv('/mnt/volume-sgp1-02/transactions.csv', parse_dates=['transaction_date','membership_expire_date'], dtype={'payment_method_id': np.int8, 'payment_plan_days': np.int16, 'plan_list_price': np.int16, 'actual_amount_paid': np.int16, 'is_auto_renew': np.int8, 'is_cancel': np.int8})

In [4]:
test = pd.read_csv("/mnt/volume-sgp1-02/sample_submission_zero.csv")

In [5]:
#map info to each user in the train set
df_train = pd.merge(train,members,how='left', on="msno")
df_train.head()

,msno,is_churn,city,bd,gender,registered_via,registration_init_time,expiration_date
0,waLDQMmcOu2jLDaV1ddDkgCrB/jl6sD66Xzs0Vqax1Y=,1,18.0,36.0,female,9.0,2005-04-06,2017-09-07
1,QA7uiXy8vIbUSPOkCf9RwQ3FsT8jVq2OxDr8zqa7bRQ=,1,10.0,38.0,male,9.0,2005-04-07,2017-03-21
2,fGwBva6hikQmTJzrbz/2Ezjm5Cth5jZUNvXigKK2AFA=,1,11.0,27.0,female,9.0,2005-10-16,2017-02-03
3,mT5V8rEpa+8wuqi6x0DoVd3H5icMKkE9Prt49UlmK+4=,1,13.0,23.0,female,9.0,2005-11-02,2017-09-26
4,XaPhtGLk/5UvvOYHcONTwsnH97P4eGECeq+BARGItRw=,1,3.0,27.0,male,9.0,2005-12-28,2017-09-27


In [6]:
transactions.head()

,msno,payment_method_id,payment_plan_days,plan_list_price,actual_amount_paid,is_auto_renew,transaction_date,membership_expire_date,is_cancel
0,YyO+tlZtAXYXoZhNr3Vg3+dfVQvrBVGO8j1mfqe4ZHc=,41,30,129,129,1,2015-09-30,2015-11-01,0
1,AZtu6Wl0gPojrEQYB8Q3vBSmE2wnZ3hi1FbK1rQQ0A4=,41,30,149,149,1,2015-09-30,2015-10-31,0
2,UkDFI97Qb6+s2LWcijVVv4rMAsORbVDT2wNXF0aVbns=,41,30,129,129,1,2015-09-30,2016-04-27,0
3,M1C56ijxozNaGD0t2h68PnH2xtx5iO5iR2MVYQB6nBI=,39,30,149,149,1,2015-09-30,2015-11-28,0
4,yvj6zyBUaqdbUQSrKsrZ+xNDVM62knauSZJzakS9OW4=,39,30,149,149,1,2015-09-30,2015-11-21,0


In [7]:
#get those with balance remaining
transactions["paid_diff"] = transactions["plan_list_price"]-transactions["actual_amount_paid"]
transactions.head(10)

,msno,payment_method_id,payment_plan_days,plan_list_price,actual_amount_paid,is_auto_renew,transaction_date,membership_expire_date,is_cancel,paid_diff
0,YyO+tlZtAXYXoZhNr3Vg3+dfVQvrBVGO8j1mfqe4ZHc=,41,30,129,129,1,2015-09-30,2015-11-01,0,0
1,AZtu6Wl0gPojrEQYB8Q3vBSmE2wnZ3hi1FbK1rQQ0A4=,41,30,149,149,1,2015-09-30,2015-10-31,0,0
2,UkDFI97Qb6+s2LWcijVVv4rMAsORbVDT2wNXF0aVbns=,41,30,129,129,1,2015-09-30,2016-04-27,0,0
3,M1C56ijxozNaGD0t2h68PnH2xtx5iO5iR2MVYQB6nBI=,39,30,149,149,1,2015-09-30,2015-11-28,0,0
4,yvj6zyBUaqdbUQSrKsrZ+xNDVM62knauSZJzakS9OW4=,39,30,149,149,1,2015-09-30,2015-11-21,0,0
5,KN7I82kjY0Tn76Ny95ncqBUVbO7b8AXrOPqZutKpxIM=,21,30,149,149,1,2015-09-30,2015-11-07,0,0
6,m5ptKif9BjdUghHXXomSezy5ohJiHm85PE13f/3kQaw=,39,30,149,149,1,2015-09-30,2015-11-28,0,0
7,uQxbyACsPOEkTIrv9jZgoGXelGBW81ZsSZKy9fhj5Z8=,39,30,149,149,1,2015-09-30,2015-11-25,0,0
8,LUPRfoE2r3WwVWhYO/TqQhjrL/qP6CO+/ORUlr7yNc0=,39,30,149,149,1,2015-09-30,2015-12-22,0,0
9,pMVjPLgVknaJYm9L0xUwxA2jmhpK7gfhGnsmmwX1mrE=,39,30,149,149,1,2015-09-30,2015-11-18,0,0


In [8]:
#aggregate information per transaction
#exclude payment_method_id first
#msno->count
#payment_plan_days,plan_list_price,actual_amount_paid,is_auto_renew,is_cancel,paid_diff -> sum
#transaction_date->min
#membership_expire_date->max
transactions_per_user = transactions.groupby(["msno"]).agg({"msno":'count',"payment_plan_days":"sum","plan_list_price":"sum","actual_amount_paid":"sum","is_auto_renew":"sum","transaction_date":"min","membership_expire_date":"max","is_cancel":"sum","paid_diff":"sum"})
transactions_per_user.head()

,is_auto_renew,plan_list_price,paid_diff,transaction_date,membership_expire_date,actual_amount_paid,msno,is_cancel,payment_plan_days
msno,,,,,,,,,
+++FOrTS7ab3tIgIh8eWwX4FqRv8w/FoiOuyXsFvphY=,0,0,0,2016-09-09,2016-09-14,0,1,0,7
+++IZseRRiQS9aaSkH6cMYU6bGDcxUieAi/tH67sC5s=,0,1788,0,2015-11-21,2017-01-04,1788,1,0,410
+++hVY1rZox/33YtvDgmKA2Frg/2qhkz12B9ylCvh8o=,4,396,0,2016-11-16,2017-03-15,396,4,0,120
+++l/EXNMLTijfLBa8p2TUVVVp2aFGSuUI/h7mLmthw=,19,2682,-149,2015-01-31,2017-03-19,2831,19,0,543
+++snpr7pmobhLKUgSHTv/mpkqgBT0tQJ0zQj6qKrqc=,26,3725,-149,2015-01-26,2017-03-26,3874,26,0,750


In [9]:
#divide auto_renew by msno_count, meaning has been always, autorenew
transactions_per_user["is_auto_renew"]=transactions_per_user["is_auto_renew"]/transactions_per_user["msno"]

In [10]:
#get how long the user has subscribed
transactions_per_user["duration"]=transactions_per_user["membership_expire_date"]-transactions_per_user["transaction_date"]
transactions_per_user.duration=transactions_per_user.duration.dt.days
transactions_per_user.head()

,is_auto_renew,plan_list_price,paid_diff,transaction_date,membership_expire_date,actual_amount_paid,msno,is_cancel,payment_plan_days,duration
msno,,,,,,,,,,
+++FOrTS7ab3tIgIh8eWwX4FqRv8w/FoiOuyXsFvphY=,0.0,0,0,2016-09-09,2016-09-14,0,1,0,7,5
+++IZseRRiQS9aaSkH6cMYU6bGDcxUieAi/tH67sC5s=,0.0,1788,0,2015-11-21,2017-01-04,1788,1,0,410,410
+++hVY1rZox/33YtvDgmKA2Frg/2qhkz12B9ylCvh8o=,1.0,396,0,2016-11-16,2017-03-15,396,4,0,120,119
+++l/EXNMLTijfLBa8p2TUVVVp2aFGSuUI/h7mLmthw=,1.0,2682,-149,2015-01-31,2017-03-19,2831,19,0,543,778
+++snpr7pmobhLKUgSHTv/mpkqgBT0tQJ0zQj6qKrqc=,1.0,3725,-149,2015-01-26,2017-03-26,3874,26,0,750,790


In [11]:
#find is_cancel!=0, means has been canceled at least once
transactions_per_user.loc[transactions_per_user["is_cancel"]>0,"is_cancel"]=1

In [12]:
transactions_per_user["msno_count"]=transactions_per_user["msno"]

In [13]:
#delete dates and msno column
del transactions_per_user["msno"]
del transactions_per_user["transaction_date"]
del transactions_per_user["membership_expire_date"]
transactions_per_user.head()

,is_auto_renew,plan_list_price,paid_diff,actual_amount_paid,is_cancel,payment_plan_days,duration,msno_count
msno,,,,,,,,
+++FOrTS7ab3tIgIh8eWwX4FqRv8w/FoiOuyXsFvphY=,0.0,0,0,0,0,7,5,1
+++IZseRRiQS9aaSkH6cMYU6bGDcxUieAi/tH67sC5s=,0.0,1788,0,1788,0,410,410,1
+++hVY1rZox/33YtvDgmKA2Frg/2qhkz12B9ylCvh8o=,1.0,396,0,396,0,120,119,4
+++l/EXNMLTijfLBa8p2TUVVVp2aFGSuUI/h7mLmthw=,1.0,2682,-149,2831,0,543,778,19
+++snpr7pmobhLKUgSHTv/mpkqgBT0tQJ0zQj6qKrqc=,1.0,3725,-149,3874,0,750,790,26


In [14]:
df_train.set_index(["msno"],inplace=True)

In [15]:
#map info to each user in the train set
df_train=df_train.join(transactions_per_user)
df_train.head()

,is_churn,city,bd,gender,registered_via,registration_init_time,expiration_date,is_auto_renew,plan_list_price,paid_diff,actual_amount_paid,is_cancel,payment_plan_days,duration,msno_count
msno,,,,,,,,,,,,,,,
waLDQMmcOu2jLDaV1ddDkgCrB/jl6sD66Xzs0Vqax1Y=,1,18.0,36.0,female,9.0,2005-04-06,2017-09-07,0.000000,149,0,149,0,37,98,2
QA7uiXy8vIbUSPOkCf9RwQ3FsT8jVq2OxDr8zqa7bRQ=,1,10.0,38.0,male,9.0,2005-04-07,2017-03-21,0.956522,3309,-149,3458,1,663,780,23
fGwBva6hikQmTJzrbz/2Ezjm5Cth5jZUNvXigKK2AFA=,1,11.0,27.0,female,9.0,2005-10-16,2017-02-03,0.800000,1492,0,1492,1,300,367,10
mT5V8rEpa+8wuqi6x0DoVd3H5icMKkE9Prt49UlmK+4=,1,13.0,23.0,female,9.0,2005-11-02,2017-09-26,0.000000,1788,0,1788,0,820,413,2
XaPhtGLk/5UvvOYHcONTwsnH97P4eGECeq+BARGItRw=,1,3.0,27.0,male,9.0,2005-12-28,2017-09-27,0.000000,3576,0,3576,0,785,787,8


In [16]:
"""we see that there are a lot of nulls for gender, so we may opt not to include it first. We also would not need registration
and expiration date.Fill other nulls with 0
"""
del df_train["gender"]
del df_train["expiration_date"]
del df_train["registration_init_time"]
df_train.fillna(0,inplace=True)

In [17]:
#delete other dfs to free up RAM
del transactions
del train

import gc
gc.collect()

362

In [18]:
df_train.head(20)

,is_churn,city,bd,registered_via,is_auto_renew,plan_list_price,paid_diff,actual_amount_paid,is_cancel,payment_plan_days,duration,msno_count
msno,,,,,,,,,,,,
waLDQMmcOu2jLDaV1ddDkgCrB/jl6sD66Xzs0Vqax1Y=,1,18.0,36.0,9.0,0.000000,149,0,149,0,37,98,2
QA7uiXy8vIbUSPOkCf9RwQ3FsT8jVq2OxDr8zqa7bRQ=,1,10.0,38.0,9.0,0.956522,3309,-149,3458,1,663,780,23
fGwBva6hikQmTJzrbz/2Ezjm5Cth5jZUNvXigKK2AFA=,1,11.0,27.0,9.0,0.800000,1492,0,1492,1,300,367,10
mT5V8rEpa+8wuqi6x0DoVd3H5icMKkE9Prt49UlmK+4=,1,13.0,23.0,9.0,0.000000,1788,0,1788,0,820,413,2
XaPhtGLk/5UvvOYHcONTwsnH97P4eGECeq+BARGItRw=,1,3.0,27.0,9.0,0.000000,3576,0,3576,0,785,787,8
GBy8qSz16X5iYWD+3CMxv/Hm6OPSrXBYtmbnlRtknW0=,1,6.0,23.0,9.0,1.000000,2384,-894,3278,1,480,700,22
lYLh7TdkWpIoQs3i3o6mIjLH8/IEgMWP9r7OpsLX0Vo=,1,13.0,29.0,9.0,0.000000,1770,0,1770,0,330,543,11
T0FF6lumjKcqEO0O+tUH2ytc+Kb9EkeaLzcVUiTr1aE=,1,11.0,22.0,9.0,0.545455,3222,-149,3371,1,633,759,22
Nb1ZGEmagQeba5E+nQj8VlQoWl+8SFmLZu+Y8ytIamw=,1,18.0,22.0,9.0,0.000000,3683,-150,3833,0,720,775,22


In [19]:
# Utility function to report best scores
from operator import itemgetter

def report_scores(grid_scores, n_top=10):
    top_scores = sorted(grid_scores, key=itemgetter(1), reverse=False)[:n_top]
    for i, score in enumerate(top_scores):
        print("Model with rank: {0}".format(i + 1))
        print("Neg Log Loss score: {0:.3f} (std: {1:.3f})".format(
              score.mean_validation_score,
              np.std(score.cv_validation_scores)))
        print("Parameters: {0}".format(score.parameters))
        print("")

In [20]:
df_class=df_train["is_churn"]
del df_train["is_churn"]

In [24]:
from sklearn.model_selection import RandomizedSearchCV
from scipy import stats
clf_xgb = xgb.XGBClassifier(objective = 'binary:logistic')
param_dist = {'n_estimators': stats.randint(150, 500),
              'learning_rate': stats.uniform(0.01, 0.07),
              'subsample': stats.uniform(0.3, 0.7),
              'max_depth': [3, 4, 5, 6, 7, 8, 9],
              'colsample_bytree': stats.uniform(0.5, 0.45),
              'min_child_weight': [1, 2, 3]
             }
random_search = RandomizedSearchCV(clf_xgb, param_distributions = param_dist, n_iter = 25, scoring = 'neg_log_loss', error_score = 0, verbose = 3)

random_search.fit(df_train, df_class)


Fitting 3 folds for each of 25 candidates, totalling 75 fits
[CV] min_child_weight=1, colsample_bytree=0.608910461469, n_estimators=254, learning_rate=0.0737693876532, max_depth=7, subsample=0.788549306762 
[CV]  min_child_weight=1, colsample_bytree=0.608910461469, n_estimators=254, learning_rate=0.0737693876532, max_depth=7, subsample=0.788549306762, score=-0.10778347871419762, total= 2.0min
[CV] min_child_weight=1, colsample_bytree=0.608910461469, n_estimators=254, learning_rate=0.0737693876532, max_depth=7, subsample=0.788549306762 


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:  2.1min remaining:    0.0s


[CV]  min_child_weight=1, colsample_bytree=0.608910461469, n_estimators=254, learning_rate=0.0737693876532, max_depth=7, subsample=0.788549306762, score=-0.1071246643334138, total= 2.0min
[CV] min_child_weight=1, colsample_bytree=0.608910461469, n_estimators=254, learning_rate=0.0737693876532, max_depth=7, subsample=0.788549306762 


[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:  4.2min remaining:    0.0s


[CV]  min_child_weight=1, colsample_bytree=0.608910461469, n_estimators=254, learning_rate=0.0737693876532, max_depth=7, subsample=0.788549306762, score=-0.10717752049945724, total= 2.1min
[CV] min_child_weight=3, colsample_bytree=0.880989853465, n_estimators=274, learning_rate=0.0266541112605, max_depth=6, subsample=0.443762759792 
[CV]  min_child_weight=3, colsample_bytree=0.880989853465, n_estimators=274, learning_rate=0.0266541112605, max_depth=6, subsample=0.443762759792, score=-0.11764934785042966, total= 2.6min
[CV] min_child_weight=3, colsample_bytree=0.880989853465, n_estimators=274, learning_rate=0.0266541112605, max_depth=6, subsample=0.443762759792 
[CV]  min_child_weight=3, colsample_bytree=0.880989853465, n_estimators=274, learning_rate=0.0266541112605, max_depth=6, subsample=0.443762759792, score=-0.11690691419162616, total= 2.4min
[CV] min_child_weight=3, colsample_bytree=0.880989853465, n_estimators=274, learning_rate=0.0266541112605, max_depth=6, subsample=0.443762759

[CV]  min_child_weight=3, colsample_bytree=0.583157819926, n_estimators=480, learning_rate=0.0160696560047, max_depth=4, subsample=0.909036417435, score=-0.12685349231890267, total= 3.0min
[CV] min_child_weight=3, colsample_bytree=0.583157819926, n_estimators=480, learning_rate=0.0160696560047, max_depth=4, subsample=0.909036417435 
[CV]  min_child_weight=3, colsample_bytree=0.583157819926, n_estimators=480, learning_rate=0.0160696560047, max_depth=4, subsample=0.909036417435, score=-0.12583551941525675, total= 2.7min
[CV] min_child_weight=3, colsample_bytree=0.583157819926, n_estimators=480, learning_rate=0.0160696560047, max_depth=4, subsample=0.909036417435 
[CV]  min_child_weight=3, colsample_bytree=0.583157819926, n_estimators=480, learning_rate=0.0160696560047, max_depth=4, subsample=0.909036417435, score=-0.125725340244668, total= 2.8min
[CV] min_child_weight=1, colsample_bytree=0.52285237646, n_estimators=205, learning_rate=0.0190363820695, max_depth=8, subsample=0.765759329807

[CV]  min_child_weight=3, colsample_bytree=0.57699651963, n_estimators=184, learning_rate=0.0328710973817, max_depth=4, subsample=0.581866612078, score=-0.1299708274100459, total= 1.0min
[CV] min_child_weight=3, colsample_bytree=0.57699651963, n_estimators=184, learning_rate=0.0328710973817, max_depth=4, subsample=0.581866612078 
[CV]  min_child_weight=3, colsample_bytree=0.57699651963, n_estimators=184, learning_rate=0.0328710973817, max_depth=4, subsample=0.581866612078, score=-0.12991420269786708, total= 1.1min
[CV] min_child_weight=2, colsample_bytree=0.786212673146, n_estimators=190, learning_rate=0.0338316669226, max_depth=5, subsample=0.517884693863 
[CV]  min_child_weight=2, colsample_bytree=0.786212673146, n_estimators=190, learning_rate=0.0338316669226, max_depth=5, subsample=0.517884693863, score=-0.12300433559569293, total= 1.6min
[CV] min_child_weight=2, colsample_bytree=0.786212673146, n_estimators=190, learning_rate=0.0338316669226, max_depth=5, subsample=0.517884693863 

[Parallel(n_jobs=1)]: Done  75 out of  75 | elapsed: 213.5min finished


RandomizedSearchCV(cv=None, error_score=0,
          estimator=XGBClassifier(base_score=0.5, colsample_bylevel=1, colsample_bytree=1,
       gamma=0, learning_rate=0.1, max_delta_step=0, max_depth=3,
       min_child_weight=1, missing=None, n_estimators=100, nthread=-1,
       objective='binary:logistic', reg_alpha=0, reg_lambda=1,
       scale_pos_weight=1, seed=0, silent=True, subsample=1),
          fit_params=None, iid=True, n_iter=25, n_jobs=1,
          param_distributions={'min_child_weight': [1, 2, 3], 'colsample_bytree': <scipy.stats._distn_infrastructure.rv_frozen object at 0x7ff148d6b828>, 'learning_rate': <scipy.stats._distn_infrastructure.rv_frozen object at 0x7ff1c9d70278>, 'n_estimators': <scipy.stats._distn_infrastructure.rv_frozen object at 0x7ff146338f28>, 'max_depth': [3, 4, 5, 6, 7, 8, 9], 'subsample': <scipy.stats._distn_infrastructure.rv_frozen object at 0x7ff148d6b668>},
          pre_dispatch='2*n_jobs', random_state=None, refit=True,
          return_train_scor

In [25]:
import gc
gc.collect()


457

In [26]:
from sklearn.externals import joblib
joblib.dump(random_search.best_estimator_, 'XGBoost1.pkl')

['XGBoost1.pkl']

In [27]:
report_scores(random_search.grid_scores_)

Model with rank: 1
Neg Log Loss score: -0.139 (std: 0.001)
Parameters: {'min_child_weight': 1, 'colsample_bytree': 0.524887060342068, 'n_estimators': 270, 'learning_rate': 0.021904494508507198, 'max_depth': 3, 'subsample': 0.73826204488312508}

Model with rank: 2
Neg Log Loss score: -0.131 (std: 0.000)
Parameters: {'min_child_weight': 1, 'colsample_bytree': 0.52285237645959826, 'n_estimators': 205, 'learning_rate': 0.01903638206947756, 'max_depth': 8, 'subsample': 0.76575932980747963}

Model with rank: 3
Neg Log Loss score: -0.130 (std: 0.000)
Parameters: {'min_child_weight': 3, 'colsample_bytree': 0.57699651963020782, 'n_estimators': 184, 'learning_rate': 0.032871097381664667, 'max_depth': 4, 'subsample': 0.58186661207832824}

Model with rank: 4
Neg Log Loss score: -0.129 (std: 0.000)
Parameters: {'min_child_weight': 1, 'colsample_bytree': 0.69707783344261376, 'n_estimators': 211, 'learning_rate': 0.015920795443086763, 'max_depth': 9, 'subsample': 0.86354129562208715}

Model with rank

/usr/local/lib/python3.5/dist-packages/sklearn/model_selection/_search.py:747: DeprecationWarning: The grid_scores_ attribute was deprecated in version 0.18 in favor of the more elaborate cv_results_ attribute. The grid_scores_ attribute will not be available from 0.20
  DeprecationWarning)


In [28]:
test.set_index(["msno"],inplace=True)

In [29]:
del test["is_churn"]
test=(test.join(transactions_per_user))

In [30]:
members.set_index(["msno"],inplace=True)

In [31]:
test=(test.join(members))

In [32]:
del test["registration_init_time"]
del test["expiration_date"]
del test["gender"]
test.head()

,is_auto_renew,plan_list_price,paid_diff,actual_amount_paid,is_cancel,payment_plan_days,duration,msno_count,city,bd,registered_via
msno,,,,,,,,,,,
ugx0CjOMzazClkFzU2xasmDZaoIqOUAZPsH1q0teWCg=,0.833333,298.0,-596.0,894.0,1.0,60.0,730.0,6.0,5.0,28.0,3.0
f/NmvEzHfhINFEYZTR05prUdr+E+3+oewvweYz9cCQE=,0.000000,627.0,0.0,627.0,0.0,127.0,374.0,5.0,13.0,20.0,3.0
zLo9f73nGGT1p21ltZC3ChiRnAVvgibMyazbCxvWPcg=,0.000000,2682.0,0.0,2682.0,0.0,540.0,566.0,18.0,13.0,18.0,3.0
8iF/+8HY8lJKFrTc7iR9ZYGCG2Ecrogbc2Vy5YhsfhQ=,0.947368,2384.0,-298.0,2682.0,1.0,487.0,790.0,19.0,1.0,0.0,7.0
K6fja4+jmoZ5xG6BypqX80Uw/XKpMgrEMdG2edFOxnA=,1.000000,2529.0,-427.0,2956.0,1.0,630.0,771.0,24.0,13.0,35.0,7.0


In [46]:
test= test[["city","bd","registered_via","is_auto_renew","plan_list_price","paid_diff","actual_amount_paid","is_cancel","payment_plan_days","duration","msno_count"]]
test.head()

,city,bd,registered_via,is_auto_renew,plan_list_price,paid_diff,actual_amount_paid,is_cancel,payment_plan_days,duration,msno_count
msno,,,,,,,,,,,
ugx0CjOMzazClkFzU2xasmDZaoIqOUAZPsH1q0teWCg=,5.0,28.0,3.0,0.833333,298.0,-596.0,894.0,1.0,60.0,730.0,6.0
f/NmvEzHfhINFEYZTR05prUdr+E+3+oewvweYz9cCQE=,13.0,20.0,3.0,0.000000,627.0,0.0,627.0,0.0,127.0,374.0,5.0
zLo9f73nGGT1p21ltZC3ChiRnAVvgibMyazbCxvWPcg=,13.0,18.0,3.0,0.000000,2682.0,0.0,2682.0,0.0,540.0,566.0,18.0
8iF/+8HY8lJKFrTc7iR9ZYGCG2Ecrogbc2Vy5YhsfhQ=,1.0,0.0,7.0,0.947368,2384.0,-298.0,2682.0,1.0,487.0,790.0,19.0
K6fja4+jmoZ5xG6BypqX80Uw/XKpMgrEMdG2edFOxnA=,13.0,35.0,7.0,1.000000,2529.0,-427.0,2956.0,1.0,630.0,771.0,24.0


In [48]:
test.fillna(0,inplace=True)
test["predict"]=random_search.best_estimator_.predict_proba(test)[:,1]


In [49]:
test["msno"]=test.index
test["is_churn"]=test.predict
test.head(40)
submission = pd.DataFrame(index=test.index)
submission.index = range(len(submission))
submission=test[["msno","is_churn"]]
submission.index = range(len(submission))
submission

,msno,is_churn
0,ugx0CjOMzazClkFzU2xasmDZaoIqOUAZPsH1q0teWCg=,0.428350
1,f/NmvEzHfhINFEYZTR05prUdr+E+3+oewvweYz9cCQE=,0.472511
2,zLo9f73nGGT1p21ltZC3ChiRnAVvgibMyazbCxvWPcg=,0.117483
3,8iF/+8HY8lJKFrTc7iR9ZYGCG2Ecrogbc2Vy5YhsfhQ=,0.928844
4,K6fja4+jmoZ5xG6BypqX80Uw/XKpMgrEMdG2edFOxnA=,0.021761
5,ibIHVYBqxGwrSExE63/omeDD99M5vYB3CN2HzkEY+eM=,0.121210
6,kVmM8X4iBPCOfK/m1l3KR7LjFoxD1EcKYCc76F5IAWw=,0.000151
7,moRTKhKIDvb+C8ZHOgmaF4dXMLk0jOn65d7a8tQ2Eds=,0.456455
8,dW/tPZMDh2Oz/ksduEctJbsz0MXw3kay/1AlZCq3EbI=,0.305345
9,otEcMhAX3mU4gumUSogqgteN3oaNmhdmTkoof2iRYEE=,0.015672


In [50]:
submission.to_csv("submission3.csv",index=False)